<a href="https://colab.research.google.com/github/kaleetock/JS002/blob/main/2021ChemEggs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Instructions

In order for this code to run correctly, you need to have added the Google spreadsheet containing the form entries from the Chemistry of Eggs experiment: "16.5: Chemistry of Eggs (Responses)" to your Google drive.  You also need to have uploaded the student roster from Canvas to your Google drive and saved it as a Google sheet.  (The assumed structure of the roster has instructor, period, first_name, preferred_name, last_name as the first, third, fourth, fifth, and sixth fields.)  

## Some assumptions 

1) Only data from 2022 and December of 2021 should be scraped from the "16.5: Chemistry of Eggs (Responses)" Google form.

2) The roster is in a Google sheet named "2022_JS002_roster".

3) A new spreadsheet named "2022 Bio Eggs" should be written for students to analyze.  (This code doesn't actually write that spreadsheet; it just outputs it.)

4) The sugar concentrations are 0, 0.42, 0.85, 1.7, and 3.4 M.

5) Students should re-do the experiment if they have fewer than 4 data points.

If any of these assumptions are incorrect, the code cell below should be edited accordingly before running.

## Authorization

Note that when you run the code cell below, you will need to authorize Google drive two times (once to read and write your Google drive files and once to read and write your Google drive spreadsheets).

In [ ]:
#@title Authorize Google Drive and Install Modules
!pip install --upgrade gspread
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os
import re
import numpy as np
import math
import bokeh.plotting as bk
from bokeh.models import Range1d
from bokeh.layouts import row

year = 2022   # This code will also use data entered in December of the previous year.
roster = "2022_JS002_roster"  # Download student roster from Canvas, upload to Gdrive, save as Google sheets, enter name here.
filename = "16.5: Chemistry of Eggs (Responses)"  # Should be able to re-use this same form from year to year.
sugar_concs = {1:0, 2:0.42, 3:0.85, 4:1.7, 5:3.4}    # Key = egg number, value = sugar conc (here concs are 0, 1/8, 1/4, 1/2, 1)
min_data_points = 4   # Any student with less than min_data_points should be asked to re-do the lab.
new_ss = "2022 Bio Eggs" # This is the new spreadsheet that will be written for the students to analyze.

Mounted at /content/drive


In [ ]:
#@title Load Students from Roster

##########################################

def get_students(roster):
  sxns, alt_names = {}, {}
  worksheet = gc.open(roster).sheet1
  rows = worksheet.get_all_values()
  row_count = 0
  for r in rows:
    row_count += 1
    if (row_count == 1):
      continue
    instructor, period, first_name, preferred_name, last_name = r[0], r[2], r[3], r[4], r[5]
    sxn = instructor+" "+period
    name = first_name+" "+last_name
    m = re.search(r"\w+", preferred_name)
    if m and not preferred_name == first_name:
      alt_name = preferred_name+" "+last_name
      alt_names[alt_name] = name
    if sxn not in sxns.keys():
      sxns[sxn] = {}
    sxns[sxn][name] = 0
  return(sxns, alt_names)

##########################################

sxns, alt_names = get_students(roster)
print(sxns)
print(alt_names)
print("\nStudents are loaded.")

{'Hamaoui, Marie P4(TTh)': {'Audrey Ginn': 0, 'Annika Hohn': 0, 'Veronica Howard': 0, 'Allison Le': 0, 'Ryan Mackeben': 0, 'Sean Tam': 0, 'Emir Tataroglu': 0}, 'Hamaoui, Marie P10(TTh)': {'Alexander Brown': 0, 'Anne Dong': 0, 'Isabella Ziye Huang': 0, 'Barbi Laguardia': 0, 'Elina Lobanov': 0, 'Aryan Poduri': 0, 'Eyob Sisay': 0, 'Kaveri Udupa': 0, 'Boshen Zhang': 0}, 'Oparko, David Christopher P8(MW)': {'David Choi': 0, 'Saya Jameson': 0, 'Sejong Kim': 0, 'Francisco Mendoza': 0, 'Zubin Patel': 0, 'Yousef Rahman': 0, 'Louis Rojas': 0, 'Adriano Walker-Neumann': 0}, 'Tock, Kalee P3(MW)': {'Jordan Bryant': 0, 'Adrian Costello': 0, 'Sage Gendreau-Distler': 0, 'Arjun Kumar': 0, 'Syl Perrin': 0, 'Fletcher Pittard': 0, 'Owen Salter': 0, 'Chloe Williams': 0}, 'Tock, Kalee P6(TTh)': {'Zoe Chong': 0, 'Henry Fultz': 0, 'Aanya Gupta': 0, 'Alexandra Headley': 0, 'CJ Horning': 0, 'Mason Jackson': 0, 'Maya Jadhav': 0, 'Chandler Nelson': 0, 'Victor Nikolov': 0, 'Rohan Raghu': 0, 'Hannah Ryou': 0, 'Maya 

In [ ]:
#@title Load Data from Chem Eggs Form

##########################################

def make_plot (name, volume, concs, pmc, sugar_concs):
  min_sugar_conc = min(sugar_concs.values())
  max_sugar_conc = max(sugar_concs.values())

  zero_x = [min_sugar_conc, max_sugar_conc]
  zero_y = [0,0]

  plot = bk.figure(
    plot_width=350, plot_height=350,  
    title = "%s: v = %s mL" % (name, volume),
    x_axis_label = "sugar concentration", y_axis_label = "Percent Mass Change"
  )
  plot.title.text_font_size = '12pt'
  plot.xaxis.axis_label_text_font_size = "12pt"
  plot.yaxis.axis_label_text_font_size = "12pt"
  plot.scatter(concs, pmc, marker="circle", size=15, color="red")
  plot.line(zero_x, zero_y, color="blue")
  return(plot)

##########################################

def get_sxn (first_name, last_name, sxns, alt_names):
  name = first_name+" "+last_name  # This is the name the student entered on the form
  alt_name = first_name.capitalize()+" "+last_name.capitalize()

  if name in alt_names.keys():
    name = alt_names[name]
  if alt_name in alt_names.keys():
    name = alt_names[alt_name]

  for sxn in sxns.keys():
    students = sxns[sxn]
    if name in students.keys():
      return(name, name, sxn)
    if alt_name in students.keys():
      return(alt_name, alt_name, sxn)
  
  print(f"{name}: could not find section!")
  return(name, alt_name, "")

##########################################

def parse_masses(mass_change_array, sugar_concs):
  num_data_points = 0
  egg_nums, concs, pmc1, pmc2, masses = [], [], [], [], {}
  egg_num = 0
  for mass in mass_change_array:
    egg_num += 1
    p1, p2, ms = parse(mass)
    masses[egg_num] = ms
    if math.isnan(p1) or math.isnan(p2):
      continue
    num_data_points += 1
    egg_nums.append(egg_num)
    concs.append(sugar_concs[egg_num])
    pmc1.append(p1)
    pmc2.append(p2)
  return(egg_nums, concs, pmc1, pmc2, num_data_points, masses)

##########################################

def parse(masses):
  ms = masses.strip().split(",")
  m = re.search(r"^\s*(\d+\.?\d*)\s*$", ms[0])
  if not m:
    return(float('nan'), float('nan'), ['N/A', 'N/A', 'N/A'])
  m1 = float(m.group(1))
  if len(ms) > 1:
    m = re.search(r"^\s*(\d+\.?\d*)\s*$", ms[1])
    if m:
      m2 = float(m.group(1))
      if len(ms) > 2:
        m = re.search(r"^\s*(\d+\.?\d*)\s*$", ms[2])
        if m:
          m3 = float(m.group(1))
          percent_mass_change_1 = round(100 * (m3 - m1)/m1)
          percent_mass_change_2 = round(100 * (m3 - m2)/m2)
          ms = [float(m1), float(m2), float(m3)]
          return (percent_mass_change_1, percent_mass_change_2, ms)

  return(float('nan'), float('nan'), ['N/A', 'N/A', 'N/A'])

##########################################

def read_egg_data_spreadsheet(filename, year, sxns, alt_names, sugar_concs):
  worksheet = gc.open(filename).sheet1
  rows = worksheet.get_all_values()
  data = {}
  row_count = 0
  year_count = 0
  for r in rows:
    row_count += 1
    if row_count == 1:
      continue
    m = re.search(r"^(\d+)\/(\d+)\/(\d+)", r[0])
    if m:
      month, day, yr = int(m.group(1)), int(m.group(2)), int(m.group(3))
      if (yr < year and not (yr == (year-1) and month == 12)):
        continue
      if re.search(r"^Tock$", r[2]) and re.search(r"^Kalee$", r[3]):
        continue
      year_count += 1
      email, volume = r[1], int(r[4])
      egg_nums, concs, pmc1, pmc2, num_data_points, masses = parse_masses([r[5], r[6], r[7], r[8], r[9]], sugar_concs)    
      name, alt_name, sxn = get_sxn(r[3], r[2], sxns, alt_names)
      sxns[sxn][name] = num_data_points
      plot = make_plot(alt_name, volume, concs, pmc1, sugar_concs)
      print(f"{name}: {num_data_points} data points: {concs} {pmc1} {sxn}")
      data[name] = {"alt_name": alt_name, "sxn": sxn, "email": email, "num_data_points": num_data_points, "plot": plot, "masses": masses }

    else:
      print(f"row {r} doesn't match!")

  print(f"\nThere are {year_count} student entries in the spreadsheet from {year}.")
  return(data)

##########################################

data = read_egg_data_spreadsheet(filename, year, sxns, alt_names, sugar_concs)


Chandler Nelson: 5 data points: [0, 0.42, 0.85, 1.7, 3.4] [44, 6, 0, -10, -23] Tock, Kalee P6(TTh)
Adrian Costello: 5 data points: [0, 0.42, 0.85, 1.7, 3.4] [58, 10, 7, -29, -37] Tock, Kalee P3(MW)
Anne Dong: 5 data points: [0, 0.42, 0.85, 1.7, 3.4] [44, 11, 0, -14, -26] Hamaoui, Marie P10(TTh)
Syl Perrin: 5 data points: [0, 0.42, 0.85, 1.7, 3.4] [62, 17, 2, -17, -20] Tock, Kalee P3(MW)
Maya Thornton: 5 data points: [0, 0.42, 0.85, 1.7, 3.4] [59, 38, 15, -14, -28] Tock, Kalee P6(TTh)
Zoe Chong: 5 data points: [0, 0.42, 0.85, 1.7, 3.4] [53, 21, -9, -9, -25] Tock, Kalee P6(TTh)
Aanya Gupta: 5 data points: [0, 0.42, 0.85, 1.7, 3.4] [45, 17, 17, -6, -17] Tock, Kalee P6(TTh)
Louis Rojas: 5 data points: [0, 0.42, 0.85, 1.7, 3.4] [52, 8, 10, -12, -33] Oparko, David Christopher P8(MW)
Boshen Zhang: 5 data points: [0, 0.42, 0.85, 1.7, 3.4] [36, 29, 29, 7, -20] Hamaoui, Marie P10(TTh)
Mason Jackson: 5 data points: [0, 0.42, 0.85, 1.7, 3.4] [28, 39, 37, 22, 14] Tock, Kalee P6(TTh)
Adriano Walker-

In [ ]:
#@title Show Data by Section

##########################################

def show_plot(p):
  try:
    bk.reset_output()
    bk.output_notebook()
    bk.show(p)
  except:
    bk.output_notebook()
    bk.show(p)

##########################################

def alphabatize_students(sxn):
  rev_names = {}
  for student in sxn.keys():
    names = student.strip().split(" ")
    first_name, last_name = names[0], names[len(names)-1]
    rev_name = last_name+","+first_name
    rev_names[rev_name] = student
  stus = []
  for rev_name in sorted(rev_names.keys()):
    stus.append(rev_names[rev_name])
  return(stus)

##########################################

def show_sxn_plots(rows):
  for row_num in sorted(rows.keys()):
    row_plots = rows[row_num]
    try:
      bk.reset_output()
      bk.output_notebook()
      bk.show(row(row_plots))
    except:
      bk.output_notebook()
      bk.show(row(row_plots))

##########################################

def write_new_ss_row(sxn, sd, ss_row, worksheet):
  name, email, masses = sd["alt_name"], sd["email"], sd["masses"]
#  worksheet.update("A"+str(ss_row), sxn)
#  worksheet.update("B"+str(ss_row), name)
#  worksheet.update("C"+str(ss_row), email)

  ls = {1:["D","E","F"], 2:["G","H","I"], 3:["J","K","L"], 4:["M","N","O"], 5:["P","Q","R"]}

  new_row = sxn+","+name+" ,"+email

  for egg_num in masses.keys():
    ms = masses[egg_num]
    l = ls[egg_num]
    for i in range(0,3):
      mass, letter = ms[i], l[i]
      if (mass == 'N/A'):
        new_row = new_row + "," + mass
#        worksheet.update(letter+str(ss_row), mass)
      else:
        new_row = new_row + "," + str("%.0f" % mass)
#        worksheet.update(letter+str(ss_row), str("%.0f" % mass))

  return(new_row)

##########################################

def analyze_data(data, sxns, min_data_points, plots_per_row, new_ss, alt_names):
#  worksheet = gc.open(new_ss).worksheet('Sheet For Students')
  worksheet = ""
  new_rows = []

  ss_row = 1
  for sxn in sxns.keys():
    print(sxn)
    students = alphabatize_students(sxns[sxn])

    rows, current_row, plot_num = {}, 1, 1
    for student in students:
      num_data_points = sxns[sxn][student]
      print(f"\t{student}: {num_data_points} data points")
      if num_data_points >= min_data_points:
        # show_plot(data[student]["plot"])
        if current_row not in rows.keys():
          rows[current_row] = []
        rows[current_row].append(data[student]["plot"])
        new_row = write_new_ss_row(sxn, data[student], ss_row, worksheet)
        new_rows.append(new_row)
#        print(new_row)
        ss_row +=1

        plot_num += 1
        if plot_num > plots_per_row:
          current_row += 1
          plot_num = 1
    show_sxn_plots(rows)
  return(new_rows)

##########################################

plots_per_row = 4
new_rows = analyze_data(data, sxns, min_data_points, plots_per_row, new_ss, alt_names)
for new_row in new_rows:
  print(new_row)


Hamaoui, Marie P4(TTh)
	Audrey Ginn: 5 data points
	Annika Hohn: 5 data points
	Veronica Howard: 5 data points
	Allison Le: 5 data points
	Ryan Mackeben: 5 data points
	Sean Tam: 5 data points
	Emir Tataroglu: 5 data points


Hamaoui, Marie P10(TTh)
	Alexander Brown: 5 data points
	Anne Dong: 5 data points
	Isabella Ziye Huang: 5 data points
	Barbi Laguardia: 0 data points
	Elina Lobanov: 4 data points
	Aryan Poduri: 5 data points
	Eyob Sisay: 4 data points
	Kaveri Udupa: 5 data points
	Boshen Zhang: 5 data points


Oparko, David Christopher P8(MW)
	David Choi: 5 data points
	Saya Jameson: 5 data points
	Sejong Kim: 4 data points
	Francisco Mendoza: 5 data points
	Zubin Patel: 0 data points
	Yousef Rahman: 0 data points
	Louis Rojas: 5 data points
	Adriano Walker-Neumann: 5 data points


Tock, Kalee P3(MW)
	Jordan Bryant: 5 data points
	Adrian Costello: 5 data points
	Sage Gendreau-Distler: 5 data points
	Arjun Kumar: 5 data points
	Syl Perrin: 5 data points
	Fletcher Pittard: 5 data points
	Owen Salter: 5 data points
	Chloe Williams: 5 data points


Tock, Kalee P6(TTh)
	Zoe Chong: 5 data points
	Henry Fultz: 5 data points
	Aanya Gupta: 5 data points
	Alexandra Headley: 5 data points
	CJ Horning: 5 data points
	Mason Jackson: 5 data points
	Maya Jadhav: 5 data points
	Chandler Nelson: 5 data points
	Victor Nikolov: 4 data points
	Rohan Raghu: 5 data points
	Hannah Ryou: 4 data points
	Maya Thornton: 5 data points


Hamaoui, Marie P4(TTh),Audrey Ginn ,audrey.a.ginn@gmail.com,61,83,91,61,81,59,62,84,69,61,80,63,59,80,46
Hamaoui, Marie P4(TTh),Annika Hohn ,annikawhohn@gmail.com,70,92,100,74,100,70,68,90,61,76,94,60,64,82,44
Hamaoui, Marie P4(TTh),Veronica Howard ,vhoward@ohs.stanford.edu,69,90,101,71,90,83,69,89,67,55,65,45,77,96,55
Hamaoui, Marie P4(TTh),Allison Le ,allison.le23@gmail.com,50,45,75,55,50,70,55,65,70,50,55,63,60,50,35
Hamaoui, Marie P4(TTh),Ryan Mackeben ,aeronawetics@gmail.com,61,86,94,58,81,81,57,84,76,58,85,62,62,88,49
Hamaoui, Marie P4(TTh),Sean Tam ,seantam@ohs.stanford.edu,63,88,95,60,80,69,60,79,70,64,88,63,61,80,48
Hamaoui, Marie P4(TTh),Emir Tataroglu ,emirtataroglu2008@gmail.com,57,78,80,59,84,52,58,73,51,57,78,47,60,80,41
Hamaoui, Marie P10(TTh),Alexander Brown ,charlesalexanderbrown08@gmail.com,62,47,99,57,30,53,66,35,75,58,30,42,61,30,43
Hamaoui, Marie P10(TTh),Anne Dong ,annedong2011@gmail.com,61,84,88,55,77,61,61,89,61,57,85,49,53,73,39
Hamaoui, Marie P10(TTh),Isabella